In [32]:
import pandas as pd
import csv
import numpy as np

In [33]:
data = pd.read_csv("w2p_cnn_final_with_triceratops.csv") 

In [34]:
data.head()

,kepid,tce_plnt_num,tce_rogue_flag,tce_period,tce_period_err,tce_time0bk,tce_time0bk_err,tce_impact,tce_impact_err,tce_duration,...,av_pp_pc,av_pp_afp,av_pp_ntp,av_training_set,av_pred_class,w2p_score,w2p_disposition,av_disposition,triceratops_probability,triceratops_disposition
0,2304168,2,0,431.719000,0.011803,219.298,0.018384,0.9308,0.03871,0.550833,...,0.0000,98.1642,1.8358,PC,AFP,0.001037,FALSE POSITIVE,FALSE POSITIVE,NaN,NaN
1,2304235,1,0,0.570607,0.000006,132.049,0.000699,0.9000,0.26870,0.048333,...,0.0014,97.8872,2.1114,PC,AFP,0.004220,FALSE POSITIVE,FALSE POSITIVE,NaN,NaN
2,2304566,1,0,0.689480,0.000048,131.833,0.019838,0.3085,45.49000,0.218167,...,0.1320,4.7366,95.1314,PC,NTP,0.003193,FALSE POSITIVE,FALSE POSITIVE,NaN,NaN
3,2304320,1,0,16.540800,0.000043,133.952,0.002193,0.8848,0.22350,0.113125,...,99.3628,0.6372,0.0000,PC,PC,0.991711,CONFIRMED,CONFIRMED,NaN,NaN
4,2303365,2,0,0.520697,0.000009,132.003,0.000916,0.9008,0.41550,0.043125,...,0.0000,99.9245,0.0755,PC,AFP,0.000136,FALSE POSITIVE,FALSE POSITIVE,NaN,NaN


In [35]:
data.columns

Index(['kepid', 'tce_plnt_num', 'tce_rogue_flag', 'tce_period',
       'tce_period_err', 'tce_time0bk', 'tce_time0bk_err', 'tce_impact',
       'tce_impact_err', 'tce_duration', 'tce_duration_err', 'tce_depth',
       'tce_depth_err', 'tce_model_snr', 'tce_prad', 'tce_prad_err', 'tce_eqt',
       'tce_eqt_err', 'tce_insol', 'tce_insol_err', 'tce_steff',
       'tce_steff_err', 'tce_slogg', 'tce_slogg_err', 'tce_sradius',
       'tce_sradius_err', 'kepoi_name', 'kepler_name', 'koi_disposition',
       'koi_pdisposition', 'koi_score', 'koi_fpflag_nt', 'koi_fpflag_ss',
       'koi_fpflag_co', 'koi_fpflag_ec', 'koi_period', 'koi_period_err1',
       'koi_period_err2', 'koi_time0bk', 'koi_time0bk_err1',
       'koi_time0bk_err2', 'koi_impact', 'koi_impact_err1', 'koi_impact_err2',
       'koi_duration', 'koi_duration_err1', 'koi_duration_err2', 'koi_depth',
       'koi_depth_err1', 'koi_depth_err2', 'koi_prad', 'koi_prad_err1',
       'koi_prad_err2', 'koi_teq', 'koi_teq_err1', 'koi_teq_err

#### Key column descriptions
- koi_disposition:  ground truth
- w2p_score: w2p probability
- w2p_disposition: w2p disposition
- koi_score: robovetter probability
- koi_pdisposition: robovetter disposition (though does not exactly match up with koi_score>0.5- unclear why)
- av_disposition: autovetter disposition (does not exactly match up with av_vf_pc>0- unclear why)
- av_vf_pc: autovetter probability (not posterior)

In [36]:
planet='PLANET'
no_planet='NO PLANET'

data.loc[data['koi_disposition']=='FALSE POSITIVE','koi_disposition']=no_planet
data.loc[data['koi_disposition']=='CONFIRMED','koi_disposition']=planet


In [37]:
data.koi_disposition.value_counts()

NO PLANET    29835
PLANET        4197
Name: koi_disposition, dtype: int64

In [38]:
data.koi_pdisposition.value_counts()

FALSE POSITIVE    29841
CANDIDATE          4191
Name: koi_pdisposition, dtype: int64

In [39]:
data.loc[data['koi_pdisposition']=='FALSE POSITIVE','koi_pdisposition']=no_planet
data.loc[data['koi_pdisposition']=='CANDIDATE','koi_pdisposition']=planet


In [40]:
data.koi_pdisposition.value_counts()

NO PLANET    29841
PLANET        4191
Name: koi_pdisposition, dtype: int64

In [41]:
data.triceratops_disposition.value_counts()

PC    253
FP    133
Name: triceratops_disposition, dtype: int64

In [42]:
data.loc[data['triceratops_disposition'] =='PC','triceratops_disposition'] = planet
data.loc[data['triceratops_disposition'] =='FP','triceratops_disposition'] = no_planet

In [43]:
data.w2p_disposition.value_counts()

FALSE POSITIVE    29943
CONFIRMED          4089
Name: w2p_disposition, dtype: int64

In [44]:
data.loc[data['w2p_disposition'] =='CONFIRMED','w2p_disposition'] = planet
data.loc[data['w2p_disposition'] =='FALSE POSITIVE','w2p_disposition'] = no_planet

In [45]:
data.w2p_disposition.value_counts()

NO PLANET    29943
PLANET        4089
Name: w2p_disposition, dtype: int64

In [46]:
data.av_disposition.value_counts()

FALSE POSITIVE    30270
CONFIRMED          3762
Name: av_disposition, dtype: int64

In [47]:
data.loc[data['av_disposition'] =='CONFIRMED','av_disposition'] = planet
data.loc[data['av_disposition'] =='FALSE POSITIVE','av_disposition'] = no_planet

In [48]:
data.av_disposition.value_counts()

NO PLANET    30270
PLANET        3762
Name: av_disposition, dtype: int64

In [49]:
for i in data.columns:
    x = data[i].unique()
    print(i)
    print(x)

kepid
[ 2304168  2304235  2304566 ... 11874338 11922290 11922648]
tce_plnt_num
[ 2  1 10  4  3  5  6  7  8  9]
tce_rogue_flag
[0 1]
tce_period
[431.719      0.570607   0.68948  ...   0.878715 114.662      1.75641 ]
tce_period_err
[1.18025e-02 5.55182e-06 4.75278e-05 ... 1.28526e-05 1.33245e-03
 1.52687e-05]
tce_time0bk
[219.298 132.049 131.833 ... 512.297 501.352 237.232]
tce_time0bk_err
[0.0183838  0.00069892 0.0198384  ... 0.00727417 0.00952692 0.00402166]
tce_impact
[0.9308   0.9      0.3085   ... 0.3957   0.1582   0.009237]
tce_impact_err
[3.871e-02 2.687e-01 4.549e+01 ... 1.850e+00 2.523e+02 6.395e-02]
tce_duration
[0.55083333 0.04833333 0.21816667 ... 1.22375    1.1825     0.35558333]
tce_duration_err
[1.144   0.1159  0.7365  ... 0.02528 0.2162  0.4014 ]
tce_depth
[ 488.4     75.15     3.794 ...  737.2    961.9   3390.   ]
tce_depth_err
[  53.7      5.45     1.923 ...   53.06   688.2   1567.   ]
tce_model_snr
[1.032e+01 2.039e+01 2.574e+00 ... 4.777e+00 5.662e+01 3.835e+03]
tce_p

In [50]:
data.columns

Index(['kepid', 'tce_plnt_num', 'tce_rogue_flag', 'tce_period',
       'tce_period_err', 'tce_time0bk', 'tce_time0bk_err', 'tce_impact',
       'tce_impact_err', 'tce_duration', 'tce_duration_err', 'tce_depth',
       'tce_depth_err', 'tce_model_snr', 'tce_prad', 'tce_prad_err', 'tce_eqt',
       'tce_eqt_err', 'tce_insol', 'tce_insol_err', 'tce_steff',
       'tce_steff_err', 'tce_slogg', 'tce_slogg_err', 'tce_sradius',
       'tce_sradius_err', 'kepoi_name', 'kepler_name', 'koi_disposition',
       'koi_pdisposition', 'koi_score', 'koi_fpflag_nt', 'koi_fpflag_ss',
       'koi_fpflag_co', 'koi_fpflag_ec', 'koi_period', 'koi_period_err1',
       'koi_period_err2', 'koi_time0bk', 'koi_time0bk_err1',
       'koi_time0bk_err2', 'koi_impact', 'koi_impact_err1', 'koi_impact_err2',
       'koi_duration', 'koi_duration_err1', 'koi_duration_err2', 'koi_depth',
       'koi_depth_err1', 'koi_depth_err2', 'koi_prad', 'koi_prad_err1',
       'koi_prad_err2', 'koi_teq', 'koi_teq_err1', 'koi_teq_err

In [51]:
#keep the star information
data['kep']=data.kepid

In [52]:
#combine star and tce information
data['kepid'] = data['kepid'].astype(str) + "_" + data['tce_plnt_num'].astype(str)
data.head()

,kepid,tce_plnt_num,tce_rogue_flag,tce_period,tce_period_err,tce_time0bk,tce_time0bk_err,tce_impact,tce_impact_err,tce_duration,...,av_pp_afp,av_pp_ntp,av_training_set,av_pred_class,w2p_score,w2p_disposition,av_disposition,triceratops_probability,triceratops_disposition,kep
0,2304168_2,2,0,431.719000,0.011803,219.298,0.018384,0.9308,0.03871,0.550833,...,98.1642,1.8358,PC,AFP,0.001037,NO PLANET,NO PLANET,NaN,NaN,2304168
1,2304235_1,1,0,0.570607,0.000006,132.049,0.000699,0.9000,0.26870,0.048333,...,97.8872,2.1114,PC,AFP,0.004220,NO PLANET,NO PLANET,NaN,NaN,2304235
2,2304566_1,1,0,0.689480,0.000048,131.833,0.019838,0.3085,45.49000,0.218167,...,4.7366,95.1314,PC,NTP,0.003193,NO PLANET,NO PLANET,NaN,NaN,2304566
3,2304320_1,1,0,16.540800,0.000043,133.952,0.002193,0.8848,0.22350,0.113125,...,0.6372,0.0000,PC,PC,0.991711,PLANET,PLANET,NaN,NaN,2304320
4,2303365_2,2,0,0.520697,0.000009,132.003,0.000916,0.9008,0.41550,0.043125,...,99.9245,0.0755,PC,AFP,0.000136,NO PLANET,NO PLANET,NaN,NaN,2303365


In [53]:
data['image'] = 'https://w210-project.s3.us-east-2.amazonaws.com/light_curve_png/' + data['kepid'].astype(str) + '.png'
data.head()

,kepid,tce_plnt_num,tce_rogue_flag,tce_period,tce_period_err,tce_time0bk,tce_time0bk_err,tce_impact,tce_impact_err,tce_duration,...,av_pp_ntp,av_training_set,av_pred_class,w2p_score,w2p_disposition,av_disposition,triceratops_probability,triceratops_disposition,kep,image
0,2304168_2,2,0,431.719000,0.011803,219.298,0.018384,0.9308,0.03871,0.550833,...,1.8358,PC,AFP,0.001037,NO PLANET,NO PLANET,NaN,NaN,2304168,https://w210-project.s3.us-east-2.amazonaws.co...
1,2304235_1,1,0,0.570607,0.000006,132.049,0.000699,0.9000,0.26870,0.048333,...,2.1114,PC,AFP,0.004220,NO PLANET,NO PLANET,NaN,NaN,2304235,https://w210-project.s3.us-east-2.amazonaws.co...
2,2304566_1,1,0,0.689480,0.000048,131.833,0.019838,0.3085,45.49000,0.218167,...,95.1314,PC,NTP,0.003193,NO PLANET,NO PLANET,NaN,NaN,2304566,https://w210-project.s3.us-east-2.amazonaws.co...
3,2304320_1,1,0,16.540800,0.000043,133.952,0.002193,0.8848,0.22350,0.113125,...,0.0000,PC,PC,0.991711,PLANET,PLANET,NaN,NaN,2304320,https://w210-project.s3.us-east-2.amazonaws.co...
4,2303365_2,2,0,0.520697,0.000009,132.003,0.000916,0.9008,0.41550,0.043125,...,0.0755,PC,AFP,0.000136,NO PLANET,NO PLANET,NaN,NaN,2303365,https://w210-project.s3.us-east-2.amazonaws.co...


In [54]:
filestack_api='Aq3TjThAiRyCfbs0Cllydz'


In [55]:
conversion_task='resize=width:600'
data['compressed_image']=('https://cdn.filestackcontent.com/'
                           +filestack_api
                           +'/'
                           +conversion_task
                           +'/'
                           + 'https://w210-project.s3.us-east-2.amazonaws.com/light_curve_png/' 
                           + data['kepid'].astype(str) 
                           + '.png')

In [66]:
conversion_task='resize=w:1200,h:600,fit:clip'
data['compressed_image_large']=('https://cdn.filestackcontent.com/'
                           +filestack_api
                           +'/'
                           +conversion_task
                           +'/'
                           + 'https://w210-project.s3.us-east-2.amazonaws.com/light_curve_png/' 
                           + data['kepid'].astype(str) 
                           + '.png')

In [67]:
data['discrepancy'] = 0
data.loc[(data['w2p_disposition']!=data['av_disposition']) | 
         (data['w2p_disposition']!=data['koi_pdisposition']) | 
         (data['koi_pdisposition']!=data['av_disposition']),
         'discrepancy'] = 1

data.loc[(data['triceratops_disposition']==planet) & (data['triceratops_disposition']!=data['av_disposition']),
         'discrepancy'] = 1

data.loc[(data['triceratops_disposition']==no_planet) & (data['triceratops_disposition']!=data['av_disposition']),
         'discrepancy'] = 1

data.loc[(data['triceratops_disposition']==planet) & (data['triceratops_disposition']!=data['koi_pdisposition']),
         'discrepancy'] = 1

data.loc[(data['triceratops_disposition']==no_planet) & (data['triceratops_disposition']!=data['koi_pdisposition']),
         'discrepancy'] = 1

data.loc[(data['triceratops_disposition']==planet) & (data['triceratops_disposition']!=data['w2p_disposition']),
         'discrepancy'] = 1

data.loc[(data['triceratops_disposition']==no_planet) & (data['triceratops_disposition']!=data['w2p_disposition']),
         'discrepancy'] = 1

data.head(100)

,kepid,tce_plnt_num,tce_rogue_flag,tce_period,tce_period_err,tce_time0bk,tce_time0bk_err,tce_impact,tce_impact_err,tce_duration,...,triceratops_disposition,kep,image,compressed_image,compressed_image_large,discrepancy,gt_w2p_disagreement,av_w2p_disagreement,rv_w2p_disagreement,t_w2p_disagreement
0,2304168_2,2,0,431.719000,0.011803,219.298,0.018384,0.9308,0.038710,0.550833,...,NaN,2304168,https://w210-project.s3.us-east-2.amazonaws.co...,https://cdn.filestackcontent.com/Aq3TjThAiRyCf...,https://cdn.filestackcontent.com/Aq3TjThAiRyCf...,0,False,False,False,False
1,2304235_1,1,0,0.570607,0.000006,132.049,0.000699,0.9000,0.268700,0.048333,...,NaN,2304235,https://w210-project.s3.us-east-2.amazonaws.co...,https://cdn.filestackcontent.com/Aq3TjThAiRyCf...,https://cdn.filestackcontent.com/Aq3TjThAiRyCf...,0,False,False,False,False
2,2304566_1,1,0,0.689480,0.000048,131.833,0.019838,0.3085,45.490000,0.218167,...,NaN,2304566,https://w210-project.s3.us-east-2.amazonaws.co...,https://cdn.filestackcontent.com/Aq3TjThAiRyCf...,https://cdn.filestackcontent.com/Aq3TjThAiRyCf...,0,False,False,False,False
3,2304320_1,1,0,16.540800,0.000043,133.952,0.002193,0.8848,0.223500,0.113125,...,NaN,2304320,https://w210-project.s3.us-east-2.amazonaws.co...,https://cdn.filestackcontent.com/Aq3TjThAiRyCf...,https://cdn.filestackcontent.com/Aq3TjThAiRyCf...,0,False,False,False,False
4,2303365_2,2,0,0.520697,0.000009,132.003,0.000916,0.9008,0.415500,0.043125,...,NaN,2303365,https://w210-project.s3.us-east-2.amazonaws.co...,https://cdn.filestackcontent.com/Aq3TjThAiRyCf...,https://cdn.filestackcontent.com/Aq3TjThAiRyCf...,0,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1433399_2,2,0,563.384000,0.018987,449.988,0.058806,0.9921,0.188300,1.396667,...,NaN,1433399,https://w210-project.s3.us-east-2.amazonaws.co...,https://cdn.filestackcontent.com/Aq3TjThAiRyCf...,https://cdn.filestackcontent.com/Aq3TjThAiRyCf...,0,False,False,False,False
96,1433399_6,6,0,57.115800,-1.000000,158.078,-1.000000,0.1000,-1.000000,0.375000,...,NaN,1433399,https://w210-project.s3.us-east-2.amazonaws.co...,https://cdn.filestackcontent.com/Aq3TjThAiRyCf...,https://cdn.filestackcontent.com/Aq3TjThAiRyCf...,0,False,False,False,False
97,1433899_1,1,0,1.041480,0.000016,131.667,0.006338,0.6718,3.088000,0.204667,...,NaN,1433899,https://w210-project.s3.us-east-2.amazonaws.co...,https://cdn.filestackcontent.com/Aq3TjThAiRyCf...,https://cdn.filestackcontent.com/Aq3TjThAiRyCf...,0,False,False,False,False
98,1433399_5,5,0,225.495000,0.010639,274.757,0.029033,0.9848,0.045350,0.536667,...,NaN,1433399,https://w210-project.s3.us-east-2.amazonaws.co...,https://cdn.filestackcontent.com/Aq3TjThAiRyCf...,https://cdn.filestackcontent.com/Aq3TjThAiRyCf...,0,False,False,False,False


In [68]:
data.discrepancy.value_counts()

0    32708
1     1324
Name: discrepancy, dtype: int64

In [69]:
#koi_disposition: ground truth
#w2p_disposition: w2p disposition
#koi_pdisposition: robovetter disposition (though does not exactly match up with koi_score>0.5- unclear why)
#av_disposition: autovetter disposition (does not exactly match up with av_vf_pc>0- unclear why)

data['gt_w2p_disagreement']=False
data['av_w2p_disagreement']=False
data['rv_w2p_disagreement']=False
data['t_w2p_disagreement']=False

data.loc[(data.koi_disposition!=data.w2p_disposition),
         'gt_w2p_disagreement'] = True

data.loc[(data.av_disposition!=data.w2p_disposition),
         'av_w2p_disagreement'] = True


data.loc[(data.koi_pdisposition!=data.w2p_disposition),
         'rv_w2p_disagreement'] = True

data.loc[(data.triceratops_disposition==planet)&(data.w2p_disposition==no_planet),
         't_w2p_disagreement'] = True

data.loc[(data.triceratops_disposition==no_planet)&(data.w2p_disposition==planet),
         't_w2p_disagreement'] = True



In [70]:
print(data.gt_w2p_disagreement.value_counts())
print(data.av_w2p_disagreement.value_counts())
print(data.rv_w2p_disagreement.value_counts())
print(data.t_w2p_disagreement.value_counts())

False    33342
True       690
Name: gt_w2p_disagreement, dtype: int64
False    33211
True       821
Name: av_w2p_disagreement, dtype: int64
False    33338
True       694
Name: rv_w2p_disagreement, dtype: int64
False    33810
True       222
Name: t_w2p_disagreement, dtype: int64


In [71]:
data.to_csv('forweb3.csv',index=False)

In [121]:
data.compressed_image_large.iloc[0]

'https://cdn.filestackcontent.com/Aq3TjThAiRyCfbs0Cllydz/resize=w:1000,h:500,fit:clip/https://w210-project.s3.us-east-2.amazonaws.com/light_curve_png/2304168_2.png'

# Break, smaller dataset below

In [13]:
data1 = data.loc[:,['kepid','koi_disposition','koi_pdisposition','tce_plnt_num','koi_score','w2p_score','w2p_disposition','av_disposition']]
data1.head()

,kepid,koi_disposition,koi_pdisposition,tce_plnt_num,koi_score,w2p_score,w2p_disposition,av_disposition
0,2304168_2,FALSE POSITIVE,FALSE POSITIVE,2,0.000,0.001037,FALSE POSITIVE,FALSE POSITIVE
1,2304235_1,FALSE POSITIVE,FALSE POSITIVE,1,0.000,0.004220,FALSE POSITIVE,FALSE POSITIVE
2,2304566_1,FALSE POSITIVE,FALSE POSITIVE,1,0.000,0.003193,FALSE POSITIVE,FALSE POSITIVE
3,2304320_1,CONFIRMED,CONFIRMED,1,0.998,0.991711,CONFIRMED,CONFIRMED
4,2303365_2,FALSE POSITIVE,FALSE POSITIVE,2,0.000,0.000136,FALSE POSITIVE,FALSE POSITIVE


In [17]:
#data1['image'] = 'https://w210-project.s3.us-east-2.amazonaws.com/light_curve_png/' + data1['kepid'].astype(str) + '_1.png'
#TRM removed the trailing _1
data1['image'] = 'https://w210-project.s3.us-east-2.amazonaws.com/light_curve_png/' + data1['kepid'].astype(str) + '.png'

data1['image'].unique()

array(['https://w210-project.s3.us-east-2.amazonaws.com/light_curve_png/2304168_2.png',
       'https://w210-project.s3.us-east-2.amazonaws.com/light_curve_png/2304235_1.png',
       'https://w210-project.s3.us-east-2.amazonaws.com/light_curve_png/2304566_1.png',
       'https://w210-project.s3.us-east-2.amazonaws.com/light_curve_png/2304320_1.png',
       'https://w210-project.s3.us-east-2.amazonaws.com/light_curve_png/2303365_2.png',
       'https://w210-project.s3.us-east-2.amazonaws.com/light_curve_png/2303102_10.png',
       'https://w210-project.s3.us-east-2.amazonaws.com/light_curve_png/2303903_1.png',
       'https://w210-project.s3.us-east-2.amazonaws.com/light_curve_png/2303365_1.png',
       'https://w210-project.s3.us-east-2.amazonaws.com/light_curve_png/2304604_1.png',
       'https://w210-project.s3.us-east-2.amazonaws.com/light_curve_png/2305255_1.png',
       'https://w210-project.s3.us-east-2.amazonaws.com/light_curve_png/757099_1.png',
       'https://w210-project.s3.

#### Filestack image size changes

https://medium.com/@WomenWhoCode/how-to-automatically-resize-fit-and-align-any-image-using-only-url-parameters-f7c81c2d3782

https://cdn.filestackcontent.com/[API-key]/[Conversion-Task]/[external-url]

In [31]:
filestack_api='Aq3TjThAiRyCfbs0Cllydz'
conversion_task='resize=width:600'

In [32]:
data1['compressed_image']=('https://cdn.filestackcontent.com/'
                           +filestack_api
                           +'/'
                           +conversion_task
                           +'/'
                           + 'https://w210-project.s3.us-east-2.amazonaws.com/light_curve_png/' 
                           + data1['kepid'].astype(str) 
                           + '.png')

In [34]:
data1.compressed_image.iloc[0]

'https://cdn.filestackcontent.com/Aq3TjThAiRyCfbs0Cllydz/resize=width:600/https://w210-project.s3.us-east-2.amazonaws.com/light_curve_png/2304168_2.png'

In [35]:
data1=data1.iloc[0:20]
print(data1)

         kepid koi_disposition koi_pdisposition  tce_plnt_num  koi_score  \
0    2304168_2  FALSE POSITIVE   FALSE POSITIVE             2      0.000   
1    2304235_1  FALSE POSITIVE   FALSE POSITIVE             1      0.000   
2    2304566_1  FALSE POSITIVE   FALSE POSITIVE             1      0.000   
3    2304320_1       CONFIRMED        CONFIRMED             1      0.998   
4    2303365_2  FALSE POSITIVE   FALSE POSITIVE             2      0.000   
5   2303102_10  FALSE POSITIVE   FALSE POSITIVE            10      0.000   
6    2303903_1       CONFIRMED        CONFIRMED             1      0.800   
7    2303365_1  FALSE POSITIVE   FALSE POSITIVE             1      0.000   
8    2304604_1  FALSE POSITIVE   FALSE POSITIVE             1      0.000   
9    2305255_1  FALSE POSITIVE   FALSE POSITIVE             1      0.000   
10    757099_1  FALSE POSITIVE   FALSE POSITIVE             1      0.000   
11    892376_1  FALSE POSITIVE   FALSE POSITIVE             1      0.000   
12    757450

In [36]:
data1.to_csv('forweb.csv',index=False)